# Machine Learning Techniques: Fixed Income ETF

In [1]:
#technical indicators

In [2]:
#libraries needed for my research 
import pandas_datareader.data as web
import datetime as dt
import math
import numpy as np
import pandas as pd
import ta
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from ta.utils import dropna
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Technical Indicators mainly for fixed income etfs (AGG, LQD, JNK)
ticker = input('Ticker: ')
end = dt.datetime.now()
start = end - dt.timedelta(days=365*15)
df = web.DataReader(ticker,'yahoo',start,end)
data = ta.add_all_ta_features(df, "Open", "High", "Low", "Close", "Volume", fillna=True).reset_index()
#shifting adjusted close one day down for forecasting reasons: I want to predict tomorrows prices using sklearn
data['forecast'] = data['Adj Close'].shift(1)
data

Ticker: HYHG


,Date,High,Low,Open,Close,Volume,Adj Close,volume_adi,volume_obv,volume_cmf,...,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,others_dr,others_dlr,others_cr,forecast
0,2013-05-23,80.019997,79.849998,79.849998,80.019997,3200.0,54.117882,3200.000000,3200.0,1.000000,...,100.000000,100.000000,-0.000000,0.000000,80.019997,0.000000,15.668072,0.000000,0.000000,NaN
1,2013-05-24,80.449997,80.169998,80.449997,80.430000,3500.0,54.395206,6200.081744,6700.0,0.925385,...,96.667218,98.333609,-3.332782,0.000000,80.216064,0.000000,0.512377,0.511068,0.512377,54.117882
2,2013-05-28,81.400002,80.230003,80.730003,80.940002,6000.0,54.740108,7482.135032,12700.0,0.589144,...,70.322698,88.996639,-29.677302,0.000000,80.550972,0.000000,0.634094,0.632092,1.149720,54.395206
3,2013-05-29,80.480003,80.150002,80.480003,80.290001,3700.0,54.300495,6921.497866,9000.0,0.422043,...,28.387198,65.125705,-71.612802,0.000000,80.439503,0.000000,-0.803066,-0.806308,0.337421,54.740108
4,2013-05-30,80.300003,80.169998,80.300003,80.300003,10900.0,54.307259,17821.497866,19900.0,0.652802,...,29.032496,42.580798,-70.967504,0.000000,80.381767,0.000000,0.012458,0.012457,0.349921,54.300495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,2020-07-06,58.709999,58.490002,58.709999,58.500000,1700.0,58.500000,907894.925933,-823700.0,-0.371366,...,44.482767,33.563267,-55.517233,-0.713853,58.818253,-2.239305,0.480932,0.479779,-26.893274,58.220001
1792,2020-07-07,59.189999,58.570000,58.680000,58.580002,7900.0,58.580002,900249.819522,-815800.0,-0.386408,...,48.239531,42.516646,-51.760469,-0.526912,58.807708,-1.925327,0.136755,0.136662,-26.793296,58.500000
1793,2020-07-08,59.049999,58.599998,58.869999,58.910000,3200.0,58.910000,901458.720467,-812600.0,-0.362816,...,59.859179,50.860492,-40.140821,-0.441853,58.810150,-0.908328,0.563329,0.561748,-26.380902,58.580002
1794,2020-07-09,59.080002,58.389999,59.080002,58.650002,4300.0,58.650002,900399.315336,-816900.0,-0.308172,...,50.704309,52.934339,-49.295691,-0.351029,58.808404,-1.196090,-0.441348,-0.442325,-26.705819,58.910000


In [4]:
data.columns

Index(['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close',
       'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi', 'momentum_mfi',
       'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_nvi', 'volume_vwap',
       'volatility_atr', 'volatility_bbm', 'volatility_bbh', 'volatility_bbl',
       'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow', 'trend_adx',
       'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
       'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
       'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv',
       '

In [5]:
#correlation matrix
correlationMatrix = data.corr()
correlationMatrix = correlationMatrix[correlationMatrix>.1]
sig = correlationMatrix['forecast'].dropna()
sig.sort_values(ascending=False)

forecast                 1.000000
Adj Close                0.995916
volume_adi               0.730983
volume_nvi               0.461123
trend_macd_signal        0.264025
trend_macd               0.255574
trend_trix               0.248083
momentum_ao              0.218893
trend_vortex_ind_pos     0.218330
trend_vortex_ind_diff    0.182151
momentum_tsi             0.174340
momentum_rsi             0.165733
trend_aroon_ind          0.130078
trend_kst                0.129508
momentum_roc             0.125747
volume_obv               0.125000
momentum_stoch_signal    0.120574
volatility_dch           0.118948
volatility_bbp           0.112433
trend_cci                0.112208
trend_kst_sig            0.106986
Name: forecast, dtype: float64

In [6]:
print("COVARIANCE MATRIX")
covarianceMatrix = data.cov()
covarianceMatrix = covarianceMatrix[covarianceMatrix > 1]
sigCov = (covarianceMatrix['forecast'].dropna())
sigCov.sort_values(ascending = False)

COVARIANCE MATRIX


volume_adi               1.300134e+06
volume_obv               1.975232e+05
Volume                   1.783671e+03
volume_fi                1.618801e+03
volume_nvi               8.813926e+01
trend_cci                4.722785e+01
trend_aroon_ind          2.840845e+01
forecast                 1.458549e+01
Adj Close                1.452604e+01
trend_kst                1.398078e+01
momentum_stoch_signal    1.251720e+01
momentum_tsi             1.244827e+01
trend_kst_sig            1.173515e+01
trend_aroon_up           1.060509e+01
momentum_wr              1.009868e+01
momentum_stoch           1.009868e+01
momentum_rsi             6.769189e+00
momentum_mfi             5.965316e+00
momentum_uo              4.006198e+00
volume_vpt               3.758096e+00
trend_kst_diff           2.245627e+00
trend_adx_pos            1.608267e+00
momentum_roc             1.106459e+00
Name: forecast, dtype: float64

In [7]:
end = dt.datetime.now()
start = end - dt.timedelta(days=365*15)
df = web.DataReader(ticker,'yahoo',start,end)
data = ta.add_all_ta_features(df, "Open", "High", "Low", "Close", "Volume", fillna=True).reset_index()
data = data[['Date','trend_ichimoku_base','volume_obv','momentum_kama','trend_ema_slow','volume_vpt','Open','High','Low','Close','Volume','Adj Close']]
#data = data[['Date','momentum_stoch','momentum_rsi','momentum_kama','trend_ema_fast','volume_vpt','Open','High','Low','Close','Volume','Adj Close']]
#data = data[['Date','volume_sma_em','trend_ichimoku_base','volume_vwap','volume_vpt','Adj Close']]
#data = data[['Date','momentum_kama','trend_ema_slow','trend_ema_fast','High','volatility_kcc','volatility_kch','trend_sma_fast','volatility_bbm','volume_vwap','trend_sma_slow','Adj Close']]             
#data = data[['Date','trend_ichimoku_b','volume_obv','momentum_kama','trend_ema_slow','volatility_kcc','Open','High','Low','Close','Volume','Adj Close']]
data['forecast'] = data['Adj Close'].shift(1)
data['Daily Return'] = (data['forecast'].pct_change())*100
data['target'] = int(False)
data.loc[data['Daily Return'] > 0, 'target']=int(True)
data

,Date,trend_ichimoku_base,volume_obv,momentum_kama,trend_ema_slow,volume_vpt,Open,High,Low,Close,Volume,Adj Close,forecast,Daily Return,target
0,2013-05-23,79.934998,3200.0,80.019997,80.019997,490.092030,79.849998,80.019997,79.849998,80.019997,3200.0,54.117882,NaN,NaN,0
1,2013-05-24,80.149998,6700.0,80.216064,80.050367,519.311481,80.449997,80.449997,80.169998,80.430000,3500.0,54.395206,54.117882,NaN,0
2,2013-05-28,80.625000,12700.0,80.550972,80.116266,55.978842,80.730003,81.400002,80.230003,80.940002,6000.0,54.740108,54.395206,0.512446,1
3,2013-05-29,80.625000,9000.0,80.439503,80.129135,8.332228,80.480003,80.480003,80.150002,80.290001,3700.0,54.300495,54.740108,0.634067,1
4,2013-05-30,80.625000,19900.0,80.381767,80.141792,-28.355568,80.300003,80.300003,80.169998,80.300003,10900.0,54.307259,54.300495,-0.803092,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,2020-07-06,59.330000,-823700.0,58.818253,58.672071,116.078149,58.709999,58.709999,58.490002,58.500000,1700.0,58.500000,58.220001,0.674389,1
1792,2020-07-07,59.330000,-815800.0,58.807708,58.665251,18.979515,58.680000,59.189999,58.570000,58.580002,7900.0,58.580002,58.500000,0.480932,1
1793,2020-07-08,59.330000,-812600.0,58.810150,58.683380,28.830187,58.869999,59.049999,58.599998,58.910000,3200.0,58.910000,58.580002,0.136755,1
1794,2020-07-09,59.330000,-816900.0,58.808404,58.680908,-0.951458,59.080002,59.080002,58.389999,58.650002,4300.0,58.650002,58.910000,0.563329,1


In [8]:
df = data.drop(['Date','Daily Return','Adj Close','forecast'], axis=1)
df = df.dropna()
df

,trend_ichimoku_base,volume_obv,momentum_kama,trend_ema_slow,volume_vpt,Open,High,Low,Close,Volume,target
0,79.934998,3200.0,80.019997,80.019997,490.092030,79.849998,80.019997,79.849998,80.019997,3200.0,0
1,80.149998,6700.0,80.216064,80.050367,519.311481,80.449997,80.449997,80.169998,80.430000,3500.0,0
2,80.625000,12700.0,80.550972,80.116266,55.978842,80.730003,81.400002,80.230003,80.940002,6000.0,1
3,80.625000,9000.0,80.439503,80.129135,8.332228,80.480003,80.480003,80.150002,80.290001,3700.0,1
4,80.625000,19900.0,80.381767,80.141792,-28.355568,80.300003,80.300003,80.169998,80.300003,10900.0,0
...,...,...,...,...,...,...,...,...,...,...,...
1791,59.330000,-823700.0,58.818253,58.672071,116.078149,58.709999,58.709999,58.490002,58.500000,1700.0,1
1792,59.330000,-815800.0,58.807708,58.665251,18.979515,58.680000,59.189999,58.570000,58.580002,7900.0,1
1793,59.330000,-812600.0,58.810150,58.683380,28.830187,58.869999,59.049999,58.599998,58.910000,3200.0,1
1794,59.330000,-816900.0,58.808404,58.680908,-0.951458,59.080002,59.080002,58.389999,58.650002,4300.0,1


In [9]:
train,test = sklearn.model_selection.train_test_split(df, test_size=0.2, random_state = 222)

In [10]:
train,val = sklearn.model_selection.train_test_split(train, test_size=0.2, random_state = 222)

In [11]:
trainy = train['target']
del train['target']
valy = val['target']
del val['target']
testy = test['target']
del test['target']

In [12]:
#Support Vector Machine 
from sklearn import svm
model1 = sklearn.svm.SVC()
model1.fit(train,trainy)
model1.score(val,valy)

0.4826388888888889

In [13]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
model2 = sklearn.ensemble.RandomForestClassifier()
model2.fit(train,trainy)
model2.score(val,valy)

0.6388888888888888

In [14]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
model3 = sklearn.linear_model.LogisticRegression()
model3.fit(train,trainy)
model3.score(val,valy)

0.6944444444444444

In [15]:
#scores of each model (generally: higher the more accurate)
mod1prob = model1.score(test,testy)
mod2prob = model2.score(test,testy)
mod3prob = model3.score(test,testy)
print(model1.score(test,testy))
print(model2.score(test,testy))
print(model3.score(test,testy))

0.5027777777777778
0.7166666666666667
0.7638888888888888


In [16]:
#columns to use below to predict tomorrow's return (whether it will be a gain (1) or a loss (0))
p1 = df[val.columns[0]].iloc[-1]
p2 = df[val.columns[1]].iloc[-1]
p3 = df[val.columns[2]].iloc[-1]
p4 = df[val.columns[3]].iloc[-1]
p5 = df[val.columns[4]].iloc[-1]
p6 = df[val.columns[5]].iloc[-1]
p7 = df[val.columns[6]].iloc[-1]
p8 = df[val.columns[7]].iloc[-1]
p9 = df[val.columns[8]].iloc[-1]
p10 = df[val.columns[9]].iloc[-1]

prediction = model3.predict([[p1,p2,p3,p4,p5,p6,p7,p8,p9,p10]])
#prediction = model3.predict([[p1,p2,p3,p4,p5,p6,p7,p8,p9]])
#prediction = model3.predict([[p1,p2,p3,p4]])
if prediction == 1:
    print(f'{ticker} => Buy: There is a {mod3prob.round(4)*100}% confidence of having a positive daily return.')
else:
    print(f'{ticker} => Sell: There is a {mod3prob.round(4)*100}% confidence of having a negative daily return.')

HYHG => Sell: There is a 76.39% confidence of having a negative daily return.


In [17]:
#confidence levels: 
#AGG => 72.58%
#JNK => 72.60%
#LQD => 76.03%
#BND => 72.16%
#HYG => 72.31%
#TLT => 75.23%

# Fixed Income ETF Screener

In [18]:
from bs4 import BeautifulSoup as bs
import requests

In [19]:
##stocks that have a 1 or 2 (buy) average recommendation from analysts usually beat the market 
#this is a little different but grouped the top rated according to ETFdb Free Trial
screener = pd.read_excel('fixed_income_etf.xlsx')
screener = screener[['Symbol','YTD Price Change','ETFdb Pro Tools']]
screener = screener.sort_values('ETFdb Pro Tools')
#gets the etfs with the highest rating of A- or higher 
etfs = screener.head(80)

In [20]:
start = '2019-01-01'
end = '2020-01-01'
failed = []
passed = []
data = pd.DataFrame()
for ticker in etfs[['Symbol']].values:
    try:
        data[ticker] = web.DataReader(ticker, data_source= "yahoo", start = start, end=end)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [21]:
#AQR study on momentum:
#suggested that securities with a positive return from the previos year usually repeats in the next year
#dataframe screener for all the stocks that have had positive returns in the past time window
data = data.dropna(axis=1)
returns = ((data.iloc[-1] - data.iloc[0])/data.iloc[0])*100
returns = pd.DataFrame(returns)
returns.columns.name = 'Ticker'
returns = returns[(returns[0] > 0)]
returns = returns.sort_values(0,ascending=False)
returns

Ticker,0
UJB,28.786511
ILTB,19.122348
BLV,18.600559
PCY,17.658411
LQD,17.005247
...,...
SCHO,3.486927
SHY,3.368372
MINT,3.335264
SHV,2.351239


In [22]:
def fundamental_metric(soup, metric):
    return soup.find(text = metric).find_next(class_='snapshot-td2').text

def get_fundamental_data(df_fund):
    for symbol in df_fund.index:
        try:
            url = ("http://finviz.com/quote.ashx?t=" + symbol)
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
            soup = bs(requests.get(url, headers=headers).content)            
            for m in df_fund.columns:                
                df_fund.loc[symbol,m] = fundamental_metric(soup,m)                
        except Exception as e:
            print (symbol, 'not found')
    return df_fund

In [23]:
stock_list = returns.index.values
metric = ['Price','Dividend','RSI (14)','SMA20','SMA50','SMA200','Perf YTD','ATR']

In [24]:
df_fund = pd.DataFrame(index=stock_list,columns=metric)
df_fund = get_fundamental_data(df_fund)
df_fund = df_fund.sort_values('RSI (14)')
df_fund['SMA20'] = df_fund['SMA20'].str.replace('%', '')
df_fund['SMA50'] = df_fund['SMA50'].str.replace('%', '')
df_fund['SMA200'] = df_fund['SMA200'].str.replace('%', '')
df_fund['Perf YTD'] = df_fund['Perf YTD'].str.replace('%', '')
df_fund = df_fund.apply(pd.to_numeric, errors='coerce')

EMSH not found


# THESE BUY/LONG AND SELL/SHORT SIGNALS ARE JUST FOR RSI

# Can put the tickers in the ML models at the top of the notebook

In [25]:
df_fund = df_fund.sort_values('RSI (14)',ascending = True).dropna(axis='index')
buy = df_fund.index.values[0:10]
sell = df_fund.index.values[-11:-1]
print(f'Possible Buy: {buy}')
print(f'Possible Sell/Short: {sell}')
df_fund

Possible Buy: ['VMBS' 'MBB' 'IBDL' 'SCHO' 'VGSH' 'SHY' 'IBDM' 'SHV' 'IBDN' 'SRLN']
Possible Sell/Short: ['MINT' 'BIV' 'MUB' 'PWZ' 'LQD' 'PZT' 'ILTB' 'BLV' 'PZA' 'ITM']


,Price,Dividend,RSI (14),SMA20,SMA50,SMA200,Perf YTD,ATR
VMBS,54.31,1.37,38.95,-0.16,-0.32,1.14,2.14,0.09
MBB,110.60,2.81,41.42,-0.16,-0.28,1.28,2.35,0.17
IBDL,25.24,0.62,43.21,-0.20,-0.25,-0.12,-0.24,0.05
SCHO,51.57,1.05,44.22,-0.04,-0.09,1.04,2.16,0.02
VGSH,62.15,1.26,45.71,-0.02,-0.07,1.08,2.17,0.03
...,...,...,...,...,...,...,...,...
ILTB,76.54,2.39,74.27,2.19,3.91,7.97,12.39,0.60
BLV,113.78,3.43,74.48,2.25,3.70,8.46,13.71,0.83
PZA,26.74,0.72,78.65,0.64,1.79,1.59,1.06,0.06
ITM,51.21,1.10,80.14,0.56,2.00,2.38,1.96,0.14


In [27]:
#RSI (14) can be a buy signal (lower numbers "usually lower than 30" indicate a buy signal bc security is oversold)
#can plug these tickers up top in the machine learning models for next trading day prediction
###EXAMPLE
buy[0]

'VMBS'